In [9]:
import sys
sys.path.append('/home/gregor/Documents/UNI/WS25/NSSC1/Schöberl/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')

from mass_spring import *
from pythreejs import *
import math
import numpy as np
from time import sleep
from IPython.display import display
from pythreejs import *


In [10]:
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

#Parameters
H = 1.0          
R = 1.0          
omega = 10.0     
mass_val = 1.0   
tilt = 0.1      

pivot = mss.add(Fix((0, 0, 0)))

mass_nodes = []
angles = [0, 2*math.pi/3, 4*math.pi/3]

for theta in angles:
    # Position
    px = R * math.cos(theta) + tilt
    py = R * math.sin(theta)
    pz = H
    m_node = mss.add(Mass(mass_val, (px, py, pz)))
    mass_nodes.append(m_node)
    
    # Velocity (Spin)
    rel_x = px - tilt
    rel_y = py
    mss.masses[m_node.nr].vel = [-omega * rel_y, omega * rel_x, 0.0]


ndof = 3 * len(mss.masses)


for m_node in mass_nodes:
    p_pivot = np.array(mss.fixes[pivot.nr].pos)
    p_mass = np.array(mss.masses[m_node.nr].pos)
    dist = np.linalg.norm(p_mass - p_pivot)
    mss.addConstraint(DistanceConstraint3d(ndof, pivot, m_node, dist, mss))


for i in range(len(mass_nodes)):
    nA = mass_nodes[i]
    nB = mass_nodes[(i + 1) % 3]
    pA = np.array(mss.masses[nA.nr].pos)
    pB = np.array(mss.masses[nB.nr].pos)
    dist = np.linalg.norm(pA - pB)
    mss.addConstraint(DistanceConstraint3d(ndof, nA, nB, dist, mss))

In [11]:
# Visual Objects 
vis_masses = [Mesh(SphereBufferGeometry(0.2), MeshStandardMaterial(color='red'), position=m.pos) for m in mss.masses]
vis_fixes = [Mesh(SphereBufferGeometry(0.2), MeshStandardMaterial(color='blue'), position=f.pos) for f in mss.fixes]

# Lines Helper Function 
def get_lines():
    pts = []
    p0 = mss.fixes[0].pos
    for m in mss.masses: pts.append([p0, m.pos])
    for i in range(3): pts.append([mss.masses[i].pos, mss.masses[(i+1)%3].pos])
    return pts

constraint_lines = LineSegments2(LineSegmentsGeometry(positions=get_lines()), LineMaterial(linewidth=2, color='black'))

#Scene & Camera
camera = PerspectiveCamera(position=[0, -6, 2], up=[0, 0, 1], aspect=600/400)
camera.lookAt([0, 0, 1])
scene = Scene(children=[*vis_masses, *vis_fixes, constraint_lines, camera, DirectionalLight(position=[10, 10, 10]), AmbientLight(intensity=0.5)])

renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], width=600, height=400)
display(renderer)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(0.0, -6.0, 2.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0,…

In [12]:
dt = 0.01
steps = 50

for i in range(1000):
    # Physics Step
    mss.simulate(dt, steps)
    
    # Graphics Update
    for m, mvis in zip(mss.masses, vis_masses):
        mvis.position = tuple(m.pos)
        
    constraint_lines.geometry = LineSegmentsGeometry(positions=get_lines())
    
    sleep(0.01)